# 🚀 02_09_HydraulicSystems_Data_Generation.ipynb

This notebook generates **synthetic hydraulic systems data** for the Databricks AeroDemo pipeline.
It creates component-level metrics focused on the aircraft’s hydraulic pressure, fluid levels, and system performance.

---

### 📋 What this notebook does:

✅ **Sets up aircraft fleet**
- Uses the same fleet IDs as earlier generators (`A320`, `B737`, `A330`)

✅ **Generates daily hydraulic system records**
- For each aircraft:
  - Simulates **hydraulic pressure** and **fluid temperature**
  - Captures **fluid level** and **pump performance**
  - Includes **line integrity** and injects random anomalies (leaks, pressure drops)

✅ **Creates a structured dataset**
- Component records → CSV for Auto Loader (`/Volumes/arao/aerodemo/tmp/hydraulic`)

✅ **Saves output safely**
- Removes old sample files before saving new ones

---

### 🛠 Key points:
- **Focus area:** Hydraulic system health, pressure regulation, and line integrity
- **Data diversity:** Includes normal, warning, and failure conditions
- **Downstream integration:** Feeds DLT tables like `twin_hydraulic_systems` and predictive models

---

### 🔗 Where this fits:
This notebook is part of the **02_ series** synthetic data generators:
- `02_01_Sensor_Data_Generation.ipynb`
- `02_02_Engine_Data_Generation.ipynb`
- `02_03_CabinPressurization_Data_Generation.ipynb`
- `02_04_Airframe_Synthetic_Data_Generation.ipynb`
- `02_05_LandingGear_Data_Generation.ipynb`
- `02_06_Avionics_Data_Generation.ipynb`
- `02_07_ElectricalSystems_Data_Generation.ipynb`
- `02_08_FuelSystems_Data_Generation.ipynb`
- `02_09_HydraulicSystems_Data_Generation.ipynb` (this file)
- Future: environmental, auxiliary, or weather systems

In [0]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random

def generate_hydraulic_systems_data(num_records_per_aircraft=100):
    aircraft_ids = ["A320_101", "A330_201", "B737_301"]
    data = {
        'hydraulic_id': [],
        'aircraft_id': [],
        'event_timestamp': [],
        'hydraulic_pressure': [],
        'fluid_temperature': [],
        'fluid_level': [],
        'pump_performance': [],
        'line_integrity': [],
        'anomaly_flag': []
    }

    for aircraft_id in aircraft_ids:
        for i in range(num_records_per_aircraft):
            random_days_ago = random.randint(0, 6)
            random_time = datetime.now() - timedelta(
                days=random_days_ago,
                hours=random.randint(0, 23),
                minutes=random.randint(0, 59)
            )
            data['hydraulic_id'].append(f'HY_{aircraft_id}_{i:03d}')
            data['aircraft_id'].append(aircraft_id)
            data['event_timestamp'].append(random_time.strftime("%Y-%m-%d %H:%M:%S"))
            data['hydraulic_pressure'].append(round(np.random.uniform(2500, 3000), 2))  # psi
            data['fluid_temperature'].append(round(np.random.uniform(-20, 80), 2))  # °C
            data['fluid_level'].append(round(np.random.uniform(50, 100), 2))  # percentage
            data['pump_performance'].append(round(np.random.uniform(0.8, 1.0), 2))  # normalized
            data['line_integrity'].append(round(np.random.uniform(0.9, 1.0), 2))  # normalized
            data['anomaly_flag'].append(random.choice(['NONE', 'WARN', 'LEAK', 'PRESSURE_DROP']))

    df = pd.DataFrame(data)
    return df

# Generate DataFrame
df = generate_hydraulic_systems_data()

# Save to Auto Loader-compatible path
output_path = "/Volumes/arao/aerodemo/tmp/hydraulic"
os.makedirs(output_path, exist_ok=True)
output_file = f"{output_path}/hydraulic_systems_sample.csv"
if os.path.exists(output_file):
    os.remove(output_file)  # safely remove old file

df.to_csv(output_file, index=False)

print(f"✅ Hydraulic systems data generated: {len(df)} rows saved to {output_file}")